# Lab One Exploring the data
## Team member
Xiaoqing Zou, Jonathan Mejia, Rongwei Ji

## Data introduction
Data source: https://www.kaggle.com/datasets/aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster 

Data file name: Lab3/data/ObesityDataSet.csv

17 colums total, 2111 rows, it contains the categorical targets



## Explanation of Task
This task is a classification problem designed to predict the degree of obesity based on given features (gender, age, height, weight, family history of obesity, smoking habits，etc). The degree of obesity can be categorized into several levels, such as Insufficient_Weight,Normal_Weight,Overweight_Level_I ,Overweight_Level_II,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III.


The business context of this task may lie in the healthcare sector, where doctors or health experts aim to assess the obesity risk of patients based on their features to develop personalized health management plans. Additionally, insurance companies might be interested as obesity can increase the risk of certain health issues, impacting insurance claim costs.


Stakeholders interested in the results include healthcare providers, health insurance companies, and individuals interested in managing their health. The model could be deployed in healthcare institutions or insurance company systems to assist decision-making or provide personalized recommendations.


## Data Preparation
For this task, we would prepare our dataset by selecting relevant features such as gender, age, height, weight, family history of obesity, smoking habits,etc. We would encode categorical variables using one-hot encoding and apply preprocessing techniques like scaling to ensure all features are on a similar scale. We may remove variables that are not useful for the analysis, such as unique identifiers or variables with too many missing values. After preprocessing, we will have a final dataset ready for classification.

## Data Splitting
We will split our dataset into training and testing sets using an 80/20 split, where 80% of the data will be used for training the model and 20% for testing its performance. This split is appropriate as it provides an adequate amount of data for training while still reserving a sizable portion for evaluation. With a larger training set, the model can learn more complex patterns in the data, leading to potentially better performance. However, it's essential to ensure that the testing set is representative of the overall data distribution to obtain reliable performance metrics.

## Modeling


In [ ]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,
                                                 self.iters)